In [7]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from utilities import plot_confusion_matrix,plot_roc_auc,plot_classification_report
from utilities import grid_search_simple,graph_result,get_classes
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import json

In [8]:
full_df = pd.read_csv("..\..\data\indeed.pre_processing.csv")

In [9]:
for col in full_df.columns:
    print(col)

Unnamed: 0.1
_id
adresse
date_de_publication
description
localisation
nom_entreprise
salaire
titre
type_de_contrat
url
Lyon
niveau_etude
type_de_cursus
type_contrat_description
grande_categorie
type_contrat_titre
Nantes
Paris
Toulouse
salaire_moyen
salaire_min
salaire_max
bac + 1
bac + 2
bac + 3
bac + 4
bac + 5
bac + 8
diplôme supérieur
ingénieur
master 1
master 2
autodidacte
ecole ingénieur
grande école
ingénieur informatique
école de commerce
école ingénieur
Apprentissage
Autre
CDD
CDI
Contrat pro
Freelance
Indépendant
Intérim
Stage
Temps partiel
Temps plein
alternance_titre
cdd_titre
cdi_titre
freelance_titre
stage_titre
3 mois renouvelable_desc
3mois renouvelable_desc
alternance_desc
cdd_desc
cdi_desc
freelance_desc
stage_desc
back-end
développeu web
développeur data
développeur mobile
développeur web
front-end
abcl
ada
afnix
agora
alf
algol
apl
applescript
asp.net
autoit
awk
back end
bash
basic
beanshell
beta
bliss
c
c#
c\+\+
cecil
charity
chuck
cilk
clean
cleo
clist
cobol
cobra
c

In [10]:
df = full_df.copy()
cols_to_drop = ["Unnamed: 0.1","adresse","date_de_publication","description","localisation","nom_entreprise","salaire",
        "titre","type_de_contrat","url","niveau_etude","type_de_cursus","type_contrat_description","grande_categorie", "type_contrat_titre"
               ,'salaire_min','salaire_max','3 mois renouvelable_desc','3mois renouvelable_desc']
df.drop(cols_to_drop, axis=1, inplace=True)

In [11]:
y = df['salaire_moyen'][pd.notnull(df['salaire_moyen'])]
X = df.drop(columns = ['salaire_moyen',"_id"])
X = X[pd.notnull(df['salaire_moyen'])]

In [12]:
y_class, dic_classe_labels,dic_min_max_mean = get_classes(y)

In [13]:
svm = SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y_class, test_size=0.25, random_state=0)

In [15]:
svm.fit(X_train,y_train)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [16]:
y_pred_reglog = svm.predict(X_test)

In [17]:
accuracy_score(y_test, y_pred_reglog)

0.706199460916442

In [18]:
df_predicted = df.drop(columns = ['salaire_moyen'])
df_predicted.reset_index(inplace=True, drop=True)
X_df_predicted = df_predicted.drop(columns = ['_id'])
X_df_predicted.reset_index(inplace=True, drop=True)

In [20]:
#%%time
y_pred_w_s = svm.predict(X_df_predicted)

In [21]:
salaire_moyen = []
salaire_min = []
salaire_max = []
for i in y_pred_w_s:
    salaire_min.append(dic_min_max_mean[i][0])
    salaire_moyen.append(dic_min_max_mean[i][1])
    salaire_max.append(dic_min_max_mean[i][2])

In [22]:
df_predicted['salaire_moyen'] = pd.DataFrame(salaire_moyen)
df_predicted['salaire_min'] = pd.DataFrame(salaire_min)
df_predicted['salaire_max'] = pd.DataFrame(salaire_max)

In [23]:
cols =["adresse","type_de_contrat","niveau_etude", "localisation","type_de_cursus"]
df_predicted[cols] = full_df[pd.isna(full_df["salaire_moyen"])][cols]

In [24]:
cols.append("salaire_moyen")
cols.append("salaire_min")
cols.append("salaire_max")
#df_predicted = df_predicted[cols]
df_predicted

,_id,Lyon,Nantes,Paris,Toulouse,bac + 1,bac + 2,bac + 3,bac + 4,bac + 5,...,zipkin,centre,salaire_moyen,salaire_min,salaire_max,adresse,type_de_contrat,niveau_etude,localisation,type_de_cursus
0,5da04bf5559bce5541bcce8f,0,0,0,1,0,0,0,0,0,...,0,1,36000.0,2600,46000,Toulouse (31),NaN,ingénieur,Toulouse,NaN
1,5da04bf5559bce5541bcce90,0,0,0,1,0,0,0,0,0,...,0,1,36000.0,2600,46000,Toulouse (31),CDI,NaN,Toulouse,NaN
2,5da04bf5559bce5541bcce91,0,0,0,1,0,0,0,0,0,...,0,0,36000.0,2600,46000,Labège (31),CDI,ingénieur,Toulouse,NaN
3,5da04bf5559bce5541bcce92,0,0,0,1,0,0,0,0,0,...,0,1,66500.0,4600,87000,Toulouse (31),CDI,ingénieur,Toulouse,autodidacte
4,5da04bf5559bce5541bcce93,0,0,0,1,0,0,0,0,0,...,0,1,36000.0,2600,46000,NaN,NaN,NaN,NaN,NaN
5,5da04bf5559bce5541bcce94,0,0,0,1,0,0,0,0,0,...,0,1,36000.0,2600,46000,Toulouse (31),NaN,ingénieur,Toulouse,NaN
6,5da04bf5559bce5541bcce95,0,0,0,1,0,0,0,0,0,...,0,1,36000.0,2600,46000,Toulouse (31),NaN,ingénieur,Toulouse,NaN
7,5da04bf5559bce5541bcce96,0,0,0,1,0,0,0,0,0,...,0,1,36000.0,2600,46000,Toulouse (31),NaN,ingénieur,Toulouse,NaN
8,5da04bf5559bce5541bcce97,0,0,1,0,0,0,0,0,0,...,0,0,13000.0,0,26000,NaN,NaN,NaN,NaN,NaN
9,5da04bf5559bce5541bcce98,0,0,1,0,0,0,0,0,0,...,0,1,66500.0,4600,87000,Paris (75),NaN,NaN,Paris,NaN


In [25]:
df_predicted.to_csv("..\..\data\indeed.predicted.csv")

In [3]:
xx =['tableau', 'powerbi','symfony', 'jquery','angular','react','react native','node js','git','github','AWS'
            'visual studio','django','flask','api rest','laravel', 'hololens', 'docker', 'jira', 'scrum', 'kanban', 
               'azure','aws', 'teamcity', 'jenkins',"oracle", "mysql","Microsoft SQL Server", "PostgreSQL","DB2","Microsoft Access",
                "SQLite","Teradata", "SAP Adaptative Server", "Hive", "FileMaker"
             ,"SAP HANA", "MariaDB", "Informix", "Firebird", "microsoft azure sql database", "Vertica", "Netezza", "Ingres", "Greenplum", "mongodb","Liquibase",
                "Buddy", "Semaphore", "GoCD", "Drone.io", "TeamCity", "Wercker", "Codeship", "Travis CI", "CircleCI", "Bamboo","Jenkins",
                "VersionOne", "Phabricator", "Asana", "Pivotal Tracker", "Basecamp", "Visual Studio", "Trello", "Jira Software", "Mingle",
                "Kallithea", "Beanstalk", "GitBucket", "Mercurial", "Gogs", "GitLab", "Bitbucket", "GitHub", "Subversion", "Git",
                "SoapUI", "Katalon Studio", "ThreatModeler", "Checkmarx", "RSpec", "SpecFlow", "Pa11y", "Browsersync",
                "Serverspec", "pytest", "BlazeMeter", "Load Impact", "Galen Framework",
                "TestNG", "QUnit", "NUnit", "FitNesse", "Karma", "Gatling", "OWASP ZAP", "Gauntlt", "Mocha", "Jmeter",
                "Cucumber", "JUnit", "Jasmine", "Selenium","Nagios","Keen IO","Opsgenie","Beats","Moogsoft","PagerDuty","Rollbar","Raygun",
                "Graphite","Grafana","APImetrics","Riemann","Atlas","Runscope","Dynatrace","Sensu","Pinpoint","Prometheus"
                ,"Vizceral","Sentry","Google Analytics","Grok","Zipkin","Zabbix","Datadog","Kibana","Elasticsearch","Logstash","Airbrake","New Relic","App Dynamics","Vector","Splunk",
                "Morpheus", "Dokku", "Engine Yard", "OpenShift", "Cloud Foundry", "Flynn", "Azure", "OpenStack",
                "Rackspace", "Google Cloud Platform", "Heroku", "Amazon Web Services","sql server"
                ]

In [4]:
import numpy as np
xx = [i.lower() for i in xx]
xx = np.unique(xx)
xx

array(['airbrake', 'amazon web services', 'angular', 'api rest',
       'apimetrics', 'app dynamics', 'asana', 'atlas', 'aws',
       'awsvisual studio', 'azure', 'bamboo', 'basecamp', 'beanstalk',
       'beats', 'bitbucket', 'blazemeter', 'browsersync', 'buddy',
       'checkmarx', 'circleci', 'cloud foundry', 'codeship', 'cucumber',
       'datadog', 'db2', 'django', 'docker', 'dokku', 'drone.io',
       'dynatrace', 'elasticsearch', 'engine yard', 'filemaker',
       'firebird', 'fitnesse', 'flask', 'flynn', 'galen framework',
       'gatling', 'gauntlt', 'git', 'gitbucket', 'github', 'gitlab',
       'gocd', 'gogs', 'google analytics', 'google cloud platform',
       'grafana', 'graphite', 'greenplum', 'grok', 'heroku', 'hive',
       'hololens', 'informix', 'ingres', 'jasmine', 'jenkins', 'jira',
       'jira software', 'jmeter', 'jquery', 'junit', 'kallithea',
       'kanban', 'karma', 'katalon studio', 'keen io', 'kibana',
       'laravel', 'liquibase', 'load impact', 'logstash

In [6]:
import pandas as pd
tt = pd.read_csv("..\..\data\indeed.pre_processing.csv")
tt.columns

Index(['Unnamed: 0.1', '_id', 'adresse', 'date_de_publication', 'description',
       'localisation', 'nom_entreprise', 'salaire', 'titre', 'type_de_contrat',
       ...
       'trello', 'vector', 'versionone', 'vertica', 'visual studio',
       'vizceral', 'wercker', 'zabbix', 'zipkin', 'centre'],
      dtype='object', length=353)